## 1. Загрузка бизнес-процесса и преобразование его в сеть Петри

In [ ]:
from pm4py import read_bpmn, convert_to_petri_net, write_xes, convert_to_dataframe, read_xes, discover_petri_net_alpha_plus
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.visualization.transition_system import visualizer as ts_visualizer
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
from pm4py.objects.conversion.process_tree import converter as pt_converter
from pm4py.algo.simulation.playout.petri_net import algorithm as simulator
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness
from pm4py.algo.conformance.tokenreplay import algorithm as conformance_diagnostics_token_based_replay
from pm4py.algo.analysis.woflan import algorithm as woflan
from pm4py.objects.petri_net.utils import reachability_graph

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

bpmn_graph = read_bpmn('lab_Vinogradov.bpmn')

net, initial_marking, final_marking = convert_to_petri_net(bpmn_graph)

print("Process Installed")
print(f"Количество переходов: {len(net.transitions)}")
print(f"Количество позиций: {len(net.places)}")
print(f"Начальная маркировка: {initial_marking}")
print(f"Конечная маркировка: {final_marking}")

gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

С ноутбука

In [ ]:
bpmn_graph = read_bpmn('lab_Vinogradov.bpmn')
net,im,fm = convert_to_petri_net(bpmn_graph)

gviz = pn_visualizer.apply(
    net,
    im,
    fm,
    parameters={"debug": True, "set_rankdir": "LR"} # set_rankdir for horizontal layout
)

# Render and view the graph
pn_visualizer.view(gviz)


## 2. Построение графа достижимости маркировок

In [ ]:
ts = reachability_graph.construct_reachability_graph(net, initial_marking)

gviz_ts = ts_visualizer.apply(ts, parameters={ts_visualizer.Variants.VIEW_BASED.value.Parameters.FORMAT: "png"})
ts_visualizer.view(gviz_ts)

print(f"Количество состояний (маркировок): {len(ts.states)}")
print(f"Количество переходов между состояниями: {len(ts.transitions)}")

In [ ]:

net, initial_marking, final_marking = convert_to_petri_net(bpmn_graph)

from pm4py.objects.petri_net.utils import reachability_graph

#конструируем граф достижимости встроенными средствами
ts=reachability_graph.construct_reachability_graph(net,initial_marking)

from pm4py.visualization.transition_system import visualizer as ts_visualizer

gviz=ts_visualizer.apply(ts,parameters={ts_visualizer.Variants.VIEW_BASED.value.Parameters.FORMAT:"png"})
ts_visualizer.view(gviz)



In [ ]:
#построения графа покрытия маркировок и графа достижимости маркировок срдествами woflan

from pm4py.algo.analysis.woflan.graphs.minimal_coverability_graph import minimal_coverability_graph
from pm4py.algo.analysis.woflan.graphs.reachability_graph import reachability_graph

from IPython.display import Image, display
from matplotlib import pyplot as plt

#граф покрытия
cg=minimal_coverability_graph.apply(net, initial_marking, final_marking)

#граф достижимости
rg=reachability_graph.apply(net, initial_marking, final_marking)


## Это максимум, которого я смог добиться, чтобы качество было максимально не шакальным

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz as pgv  # можно установить без использования напрямую

def vizualizeGraph_graphviz(network):
    """Использование Graphviz layout через NetworkX"""
    is_bounded = True

    # Проверка на неограниченность
    for node, attributes in network.nodes(data=True):
        marking = attributes.get('marking', [])
        for pos in marking:
            if pos == float('inf'):
                is_bounded = False

    # Создаем граф AGraph для использования Graphviz
    A = to_agraph(network)

    # Настраиваем атрибуты как в исходном коде
    A.graph_attr['rankdir'] = 'LR'
    A.node_attr['shape'] = 'ellipse'

    # Устанавливаем метки узлов
    for node in network.nodes():
        marking = network.nodes[node].get('marking', '')
        n = A.get_node(node)
        n.attr['label'] = str(marking)

    # Устанавливаем метки ребер
    for u, v, data in network.edges(data=True):
        e = A.get_edge(u, v)
        e.attr['label'] = data.get('transition', '')

    # Визуализация
    A.draw('graph.png', prog='dot', format='png')
    display(Image('graph.png'))

    return is_bounded

In [ ]:
vizualizeGraph_graphviz(cg)

In [ ]:
vizualizeGraph_graphviz(rg)

In [ ]:
from pm4py.objects.petri_net.utils import petri_utils
from pm4py.objects.petri_net.exporter import exporter as pnml_exporter
from pm4py.visualization.petri_net import visualizer as pn_visualizer

from pm4py.objects.petri_net.obj import PetriNet, Marking



import copy


# делаем копию сети
net_copy = copy.deepcopy(net)



# добавляем X - переход и проверяем на ограниченность

def add_cycle_transition(pn: PetriNet, first_place: PetriNet.Place, last_place: PetriNet.Place):
    """
    Добавляет переход, который берёт токен из последнего места
    и кладёт его в первое место.
    """
    # создаём переход
    t_cycle = PetriNet.Transition("t_cycle", "t_cycle")

    # добавляем переход в сеть
    pn.transitions.add(t_cycle)

    # дуги: last_place → t_cycle → first_place
    petri_utils.add_arc_from_to(last_place, t_cycle, pn)
    petri_utils.add_arc_from_to(t_cycle, first_place, pn)
    return t_cycle


#вычисляем начальные и конечные позиции
begin_places = [place for place, tokens in im.items() if tokens > 0]

final_places = [place for place, tokens in fm.items() if tokens > 0]

# создаём переход
t_cycle = PetriNet.Transition("t_cycle", "t_cycle")

# добавляем переход в сеть
net_copy.transitions.add(t_cycle)


#получить реальную позицию (объект) по ключу из маркировки

def get_place_by_name(pn, name):
    for place in pn.places:
        if place.name == name:
            return place
    return None  # если места с таким именем нет




p_fin = next(iter(fm.keys()))   # объект Place
p_beg = next(iter(im.keys()))   # объект Place
p_Fin = get_place_by_name (net_copy, p_fin.name)
p_Beg = get_place_by_name (net_copy, p_beg.name)


#добавляем дуги
petri_utils.add_arc_from_to(p_Fin, t_cycle, net_copy)
petri_utils.add_arc_from_to( t_cycle,p_Beg, net_copy)


#покажем расширенную сетку
gviz = pn_visualizer.apply(
    net_copy,
    im,
    fm,
    parameters={"debug": True, "set_rankdir": "LR"} # set_rankdir for horizontal layout
)

# отобразим
pn_visualizer.view(gviz)


# строим граф покрытия маркировок расширенной сети  и там же проверяем на ограниченность (нет омег)

cg_ext=minimal_coverability_graph.apply(net_copy, initial_marking, final_marking)
is_bounded=vizualizeGraph_graphviz(cg_ext)
print("Is bounded?:",is_bounded)

## 3. Анализ бездефектности (Woflan) и дополнительные проверки

In [ ]:
is_sound_result = woflan.apply(net, initial_marking, final_marking, 
                                parameters={
                                    woflan.Parameters.RETURN_ASAP_WHEN_NOT_SOUND: True,
                                    woflan.Parameters.PRINT_DIAGNOSTICS: True,
                                    woflan.Parameters.RETURN_DIAGNOSTICS: True
                                })

print(f"Сеть Петри является корректной (бездефектной): {is_sound_result[0]}")

# Проверка ограниченности сети через граф достижимости
print(f"1. Проверка ограниченности:")
print(f"Количество состояний в графе достижимости: {len(ts.states)}")

if len(ts.states) < 1000:
    print(f"Сеть ограничена (конечное число состояний)")
    is_bounded = True
else:
    print(f"Сеть может быть неограниченной")
    is_bounded = False

# Построение расширенной сети для проверки живости
print(f"\n2. Проверка живости сети:")

# Преобразуем граф достижимости в NetworkX для анализа сильносвязных компонент
try:
    G = nx.DiGraph()
    
    for state in ts.states:
        G.add_node(str(state))
    
    for transition in ts.transitions:
        G.add_edge(str(transition.from_state), str(transition.to_state))
    
    # Находим сильносвязные компоненты (алгоритм Косарайю)
    scc = list(nx.strongly_connected_components(G))
    
    print(f"Найдено {len(scc)} сильносвязных компонент")
    
    all_transitions = set([t.name for t in net.transitions])
    
    # Для живости сети должна быть одна компонента, содержащая все переходы
    if len(scc) == 1:
        print(f"раф достижимости сильносвязный")
        is_live = True
    else:
        # Проверяем, есть ли компонента, которая достижима из всех других
        print(f"Граф достижимости не является сильносвязным")
        
        # Находим компоненты, которые являются стоками (терминальными SCC)
        condensation = nx.condensation(G)
        terminal_scc = [node for node in condensation.nodes() if condensation.out_degree(node) == 0]
        
        if len(terminal_scc) == 1:
            print(f"Найдена одна терминальная SCC")
            is_live = True
        else:
            print(f"Найдено {len(terminal_scc)} терминальных SCC")
            is_live = False
    
    print(f"\n3. Проверка по теореме о бездефектности:")
    print(f"Сеть ограничена: {is_bounded}")
    print(f"Сеть живая: {is_live}")
    
    if is_bounded and is_live:
        print(f"ВЫВОД: Сеть является бездефектной (по теореме)")
    else:
        print(f"ВЫВОД: Сеть не является бездефектной (по теореме)")
        
except Exception as e:
    print(f"Ошибка при анализе SCC: {e}")
    print("Продолжаем анализ через другие методы")

In [ ]:
net, initial_marking, final_marking = convert_to_petri_net(bpmn_graph)

from pm4py.algo.analysis.woflan import algorithm as woflan

is_sound = woflan.apply(net, initial_marking, final_marking, parameters={woflan.Parameters.RETURN_ASAP_WHEN_NOT_SOUND: True,
                                                     woflan.Parameters.PRINT_DIAGNOSTICS: True,

                                                     woflan.Parameters.RETURN_DIAGNOSTICS: True})

print("\nIs Workflow net sound? ->", is_sound[0])

print("\n Полная статистика",is_sound[1])


## 4. Генерация журнала событий

In [ ]:
simulated_log = simulator.apply(net, initial_marking,
                                variant=simulator.Variants.BASIC_PLAYOUT,
                                parameters={simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: 50})

# Сохранение сгенерированного журнала
write_xes(simulated_log, 'generated_log.xes')

dataframe = convert_to_dataframe(simulated_log)
dataframe.to_csv('generated_log.csv', index=False)

print(f"Количество трасс: {len(simulated_log)}")
print(f"Общее количество событий: {len(dataframe)}")

In [ ]:
simulated_log = simulator.apply(net, initial_marking, variant=simulator.Variants.BASIC_PLAYOUT,
                                    parameters={simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: 50})

write_xes(simulated_log, 'log.xes')

gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

dataframe = convert_to_dataframe(simulated_log)
dataframe.to_csv('exp.csv')


In [ ]:
df = pd.read_csv("exp.csv", index_col=0)
df = df.rename(columns={"case:concept:name": "client", "time:timestamp": "datetime", "concept:name": "action"})
df['resource']=''
df.head(20)

## 5. Предварительный анализ журнала событий

In [ ]:
df = pd.read_csv("generated_log.csv")

# Переименование столбцов для удобства
df = df.rename(columns={
    "case:concept:name": "case_id",
    "time:timestamp": "datetime",
    "concept:name": "activity"
})

# Преобразование даты
df['datetime'] = pd.to_datetime(df['datetime'])

# Определение ресурсов для каждого действия на основе BPMN процесса
applicant_activities = ['Получение разрешения заявителем', 'Исправление замечаний по проекту']
clerk_activities = ['Первичная проверка документов', 'Рассмотрение заместителем руководителя']
architect_activities = ['Проверка архитектором-экспертом', 'Ручная проверка архитектора-эксперта']
engineer_activities = ['Проверка инженером-экспертом', 'Проверка инженером экспертом']
fire_service_activities = ['Запрос заключения пожарного эксперта', 'Ручная проверка пожарного надзора']
sanepidemic_activities = ['Запрос заключения санитарно-эпидемиологической службы',
                          'Проверка санэпидем надзором', 'Медкомиссия']
supervisor_activities = ['Рассмотрение руководителем']

# Назначение ресурсов
df['resource'] = 'Система/Процесс'  # Значение по умолчанию

for idx, row in df.iterrows():
    activity = str(row['activity'])

    if any(act in activity for act in applicant_activities):
        df.at[idx, 'resource'] = 'Заявитель'
    elif any(act in activity for act in clerk_activities):
        df.at[idx, 'resource'] = 'Служащий'
    elif any(act in activity for act in architect_activities):
        df.at[idx, 'resource'] = 'Архитектор'
    elif any(act in activity for act in engineer_activities):
        df.at[idx, 'resource'] = 'Инженер'
    elif any(act in activity for act in fire_service_activities):
        df.at[idx, 'resource'] = 'Пожарный надзор'
    elif any(act in activity for act in sanepidemic_activities):
        df.at[idx, 'resource'] = 'Санэпидемнадзор'
    elif any(act in activity for act in supervisor_activities):
        df.at[idx, 'resource'] = 'Руководитель'

print("Журнал событий обработан")
print(f"Размерность данных: {df.shape[0]} строк, {df.shape[1]} столбцов")
print(f"\nРаспределение событий по ресурсам:")
print(df['resource'].value_counts())

# Сохранение обработанного журнала
df.to_csv('processed_log.csv', index=False)
print("\nОбработанный журнал сохранен в 'processed_log.csv'")

## 6. Временной анализ журнала событий

In [ ]:
events = df.copy()

case_starts_ends = events.groupby('case_id')['datetime'].agg(['min', 'max']).reset_index()
case_starts_ends.columns = ['case_id', 'casestart', 'caseend']

events = events.merge(case_starts_ends, on='case_id')

# Расчет относительного времени
events['relativetime'] = events['caseend'] - events['casestart']
events['relativetime_s'] = events['relativetime'].dt.total_seconds()
events['relativedays'] = events['relativetime'].dt.days

# Дополнительные временные характеристики
events['weekday'] = events['datetime'].dt.weekday
events['date'] = events['datetime'].dt.date
events['startdate'] = events['casestart'].dt.date
events['hour'] = events['datetime'].dt.hour

print(f"\nСтатистика длительности кейсов:")
case_durations = events.groupby('case_id')['relativetime_s'].max()
print(f"Минимальная длительность: {case_durations.min():.0f} сек ({case_durations.min()/60:.1f} мин)")
print(f"Максимальная длительность: {case_durations.max():.0f} сек ({case_durations.max()/60:.1f} мин)")
print(f"Средняя длительность: {case_durations.mean():.0f} сек ({case_durations.mean()/60:.1f} мин)")
print(f"Медианная длительность: {case_durations.median():.0f} сек ({case_durations.median()/60:.1f} мин)")

plt.figure(figsize=(10, 6))
sns.histplot(case_durations, bins=20, kde=True)
plt.title('Распределение длительности кейсов')
plt.xlabel('Длительность (секунды)')
plt.ylabel('Количество кейсов')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Визуализация журнала событий

## 8. Реконструкция процесса алгоритмом Alpha Miner

In [ ]:
log_for_mining = read_xes('generated_log.xes')

net_alpha, im_alpha, fm_alpha = alpha_miner.apply(log_for_mining)

parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
gviz_alpha = pn_visualizer.apply(net_alpha, im_alpha, fm_alpha,
                                 parameters=parameters,
                                 variant=pn_visualizer.Variants.FREQUENCY,
                                 log=log_for_mining)
pn_visualizer.view(gviz_alpha)

print(f"Реконструированная сеть Петри:")
print(f"  - Переходов: {len(net_alpha.transitions)}")
print(f"  - Позиций: {len(net_alpha.places)}")

## 9. Реконструкция процесса алгоритмом Alpha+ Miner

In [ ]:
net_alpha_plus, im_alpha_plus, fm_alpha_plus = discover_petri_net_alpha_plus(log_for_mining)

# Визуализация
gviz_alpha_plus = pn_visualizer.apply(net_alpha_plus, im_alpha_plus, fm_alpha_plus,
                                      parameters=parameters,
                                      variant=pn_visualizer.Variants.FREQUENCY,
                                      log=log_for_mining)
pn_visualizer.view(gviz_alpha_plus)

print(f"Реконструированная сеть Петри:")
print(f"  - Переходов: {len(net_alpha_plus.transitions)}")
print(f"  - Позиций: {len(net_alpha_plus.places)}")

## 10. Реконструкция процесса алгоритмом Heuristics Miner

In [ ]:

heu_net = heuristics_miner.apply_heu(log_for_mining)

gviz_heu = hn_visualizer.apply(heu_net)
hn_visualizer.view(gviz_heu)

net_heu, im_heu, fm_heu = heuristics_miner.apply(log_for_mining)

gviz_heu_pn = pn_visualizer.apply(net_heu, im_heu, fm_heu)
pn_visualizer.view(gviz_heu_pn)

print(f"Сеть Петри из Heuristics Miner:")
print(f"  - Переходов: {len(net_heu.transitions)}")
print(f"  - Позиций: {len(net_heu.places)}")

## 11. Реконструкция процесса алгоритмом Inductive Miner

In [ ]:

tree = inductive_miner.apply(log_for_mining)

gviz_tree = pt_visualizer.apply(tree)
pt_visualizer.view(gviz_tree)

net_ind, im_ind, fm_ind = pt_converter.apply(tree)

print(f"Сеть Петри из Inductive Miner:")
print(f"  - Переходов: {len(net_ind.transitions)}")
print(f"  - Позиций: {len(net_ind.places)}")

## 12. Оценка качества реконструкции (Token-based Replay)

In [ ]:

replayed_traces_alpha = conformance_diagnostics_token_based_replay.apply(
    log_for_mining, net_alpha, im_alpha, fm_alpha)

# Оценка качества для Heuristics Miner
replayed_traces_heu = conformance_diagnostics_token_based_replay.apply(
    log_for_mining, net_heu, im_heu, fm_heu)

# Оценка качества для Inductive Miner
replayed_traces_ind = conformance_diagnostics_token_based_replay.apply(
    log_for_mining, net_ind, im_ind, fm_ind)

def analyze_replay_results(replayed_traces, algorithm_name):
    total_traces = len(replayed_traces)
    fitting_traces = sum(1 for trace in replayed_traces if trace.get('trace_is_fit', False))
    percent_fitting = (fitting_traces / total_traces * 100) if total_traces > 0 else 0
    
    # Среднее количество недостающих токенов
    missing_tokens = np.mean([trace.get('missing_tokens', 0) for trace in replayed_traces])
    # Среднее количество оставшихся токенов
    remaining_tokens = np.mean([trace.get('remaining_tokens', 0) for trace in replayed_traces])
    # Среднее количество продуцированных токенов
    produced_tokens = np.mean([trace.get('produced_tokens', 0) for trace in replayed_traces])
    
    return {
        'algorithm': algorithm_name,
        'total_traces': total_traces,
        'fitting_traces': fitting_traces,
        'percent_fitting': percent_fitting,
        'missing_tokens': missing_tokens,
        'remaining_tokens': remaining_tokens,
        'produced_tokens': produced_tokens
    }

results = [
    analyze_replay_results(replayed_traces_alpha, "Alpha Miner"),
    analyze_replay_results(replayed_traces_heu, "Heuristics Miner"),
    analyze_replay_results(replayed_traces_ind, "Inductive Miner")
]

for result in results:
    print(f"\n{result['algorithm']}:")
    print(f"Соответствующих трасс: {result['fitting_traces']}/{result['total_traces']} ({result['percent_fitting']:.1f}%)")
    print(f"Среднее недостающих токенов: {result['missing_tokens']:.2f}")
    print(f"Среднее оставшихся токенов: {result['remaining_tokens']:.2f}")
    print(f"Среднее продуцированных токенов: {result['produced_tokens']:.2f}")

## 13. Вычисление метрик качества реконструкции (Fitness)

In [ ]:

log_fitness_alpha = replay_fitness.evaluate(replayed_traces_alpha,
                                            variant=replay_fitness.Variants.TOKEN_BASED)
log_fitness_heu = replay_fitness.evaluate(replayed_traces_heu,
                                          variant=replay_fitness.Variants.TOKEN_BASED)
log_fitness_ind = replay_fitness.evaluate(replayed_traces_ind,
                                          variant=replay_fitness.Variants.TOKEN_BASED)

print("=== Метрики Fitness (соответствие модели журналу) ===")

fitness_summary = pd.DataFrame({
    'Алгоритм': ['Alpha Miner', 'Heuristics Miner', 'Inductive Miner'],
    'fitness': [
        log_fitness_alpha.get('average_trace_fitness', 0),
        log_fitness_heu.get('average_trace_fitness', 0),
        log_fitness_ind.get('average_trace_fitness', 0)
    ],
    'percentage_fit_traces': [
        log_fitness_alpha.get('percentage_of_fitting_traces', 0),
        log_fitness_heu.get('percentage_of_fitting_traces', 0),
        log_fitness_ind.get('percentage_of_fitting_traces', 0)
    ],
    'log_fitness': [
        log_fitness_alpha.get('log_fitness', 0),
        log_fitness_heu.get('log_fitness', 0),
        log_fitness_ind.get('log_fitness', 0)
    ]
})

print(fitness_summary.to_string(index=False))

plt.figure(figsize=(10, 6))
x_pos = np.arange(len(fitness_summary))
width = 0.25

bars1 = plt.bar(x_pos - width, fitness_summary['fitness'], width,
                label='Fitness (среднее)', color='skyblue')
bars2 = plt.bar(x_pos, fitness_summary['percentage_fit_traces']/100, width,
                label='% подходящих трасс', color='lightgreen')
bars3 = plt.bar(x_pos + width, fitness_summary['log_fitness'], width,
                label='Log Fitness', color='salmon')

plt.title('Сравнение метрик качества реконструкции', fontsize=14, pad=15)
plt.xlabel('Алгоритм')
plt.ylabel('Значение метрики')
plt.xticks(x_pos, fitness_summary['Алгоритм'])
plt.legend()
plt.grid(axis='y', alpha=0.3)

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

best_idx = fitness_summary['fitness'].idxmax()
best_algorithm = fitness_summary.loc[best_idx, 'Алгоритм']
best_fitness = fitness_summary.loc[best_idx, 'fitness']

print(f"\nВЫВОД: Наилучший результат показал {best_algorithm} с fitness = {best_fitness:.3f}")

## 14. Дополнительный анализ: построение дерева покрытия маркировок расширенной сети

In [ ]:

print("=== Дополнительный анализ: дерево покрытия маркировок расширенной сети ===")

try:
    # Импортируем необходимые функции
    from pm4py.objects.petri_net.utils.petri_utils import add_arc_from_to

    # Создаем копию сети для экспериментов
    from pm4py.objects.petri_net.obj import PetriNet, Marking

    # Создаем расширенную сеть (добавляем переход из конечной в начальную позицию)
    extended_net = PetriNet()

    # Копируем все элементы из исходной сети
    place_map = {}
    for place in net.places:
        new_place = PetriNet.Place(place.name)
        extended_net.places.add(new_place)
        place_map[place] = new_place

    transition_map = {}
    for transition in net.transitions:
        new_transition = PetriNet.Transition(transition.name, transition.label)
        extended_net.transitions.add(new_transition)
        transition_map[transition] = new_transition

    # Копируем дуги
    for arc in net.arcs:
        if arc.source in place_map and arc.target in transition_map:
            add_arc_from_to(place_map[arc.source], transition_map[arc.target], extended_net)
        elif arc.source in transition_map and arc.target in place_map:
            add_arc_from_to(transition_map[arc.source], place_map[arc.target], extended_net)

    # Создаем начальную маркировку для расширенной сети
    extended_im = Marking()
    for place, marking in initial_marking.items():
        if place in place_map:
            extended_im[place_map[place]] = marking

    # Добавляем дополнительный переход из конечной в начальную позицию
    # Находим начальную и конечную позиции
    start_place = None
    end_place = None

    # Сначала ищем по ключевым словам в именах
    for place in extended_net.places:
        place_name_lower = place.name.lower()
        if any(keyword in place_name_lower for keyword in ['start', 'начал', 'источник', 'source']):
            start_place = place
        if any(keyword in place_name_lower for keyword in ['end', 'конеч', 'финиш', 'sink']):
            end_place = place

    # Если не нашли по имени, пробуем другие стратегии
    if start_place is None:
        # Ищем позиции, которые есть в начальной маркировке
        for place in extended_net.places:
            if place in extended_im:
                start_place = place
                break

    if end_place is None:
        # Ищем позиции, которые имеют много исходящих дуг
        max_outgoing = 0
        for place in extended_net.places:
            outgoing_count = sum(1 for arc in extended_net.arcs if arc.source == place)
            if outgoing_count > max_outgoing:
                max_outgoing = outgoing_count
                end_place = place

    # Если все еще не нашли, берем первую и последнюю
    if start_place is None:
        start_place = list(extended_net.places)[0]
    if end_place is None:
        end_place = list(extended_net.places)[-1]

    print(f"Выбраны позиции для добавления reset перехода:")
    print(f"  - Начальная позиция: {start_place.name}")
    print(f"  - Конечная позиция: {end_place.name}")

    # Создаем переход "reset"
    reset_transition = PetriNet.Transition("reset_transition", "reset")
    extended_net.transitions.add(reset_transition)

    # Добавляем дуги: end_place -> reset_transition -> start_place
    add_arc_from_to(end_place, reset_transition, extended_net)
    add_arc_from_to(reset_transition, start_place, extended_net)

    print(f"\nРасширенная сеть создана:")
    print(f"  - Переходов: {len(extended_net.transitions)} (включая reset)")
    print(f"  - Позиций: {len(extended_net.places)}")
    print(f"  - Добавлен переход 'reset' из {end_place.name} в {start_place.name}")

    # Визуализируем расширенную сеть
    gviz_extended = pn_visualizer.apply(extended_net, extended_im, Marking())
    pn_visualizer.view(gviz_extended)

    # Строим граф достижимости для расширенной сети
    print("\nСтроим граф достижимости для расширенной сети...")
    extended_ts = reachability_graph.construct_reachability_graph(extended_net, extended_im)

    print(f"\nГраф достижимости расширенной сети:")
    print(f"  - Количество состояний: {len(extended_ts.states)}")
    print(f"  - Количество переходов: {len(extended_ts.transitions)}")

    # Визуализируем граф достижимости
    gviz_extended_ts = ts_visualizer.apply(
        extended_ts,
        parameters={ts_visualizer.Variants.VIEW_BASED.value.Parameters.FORMAT: "png"}
    )
    ts_visualizer.view(gviz_extended_ts)

    # Проверяем, является ли граф достижимости конечным (ограниченность)
    if len(extended_ts.states) < 1000:
        print(f"  - Граф достижимости конечный -> сеть ограничена")

        # Преобразуем граф достижимости в NetworkX для анализа сильносвязных компонент
        G_extended = nx.DiGraph()

        # Добавляем узлы (состояния)
        for state in extended_ts.states:
            G_extended.add_node(str(state))

        # Добавляем ребра (переходы)
        for transition in extended_ts.transitions:
            G_extended.add_edge(str(transition.from_state), str(transition.to_state))

        # Находим сильносвязные компоненты с помощью алгоритма Косарайю
        scc_extended = list(nx.strongly_connected_components(G_extended))

        print(f"\nАнализ сильносвязных компонент (SCC):")
        print(f"  - Количество SCC: {len(scc_extended)}")

        # Выводим размеры компонент
        for i, component in enumerate(scc_extended, 1):
            print(f"    SCC {i}: {len(component)} состояний")

        # Проверяем условия живости
        if len(scc_extended) == 1:
            print(f"  - Граф достижимости сильносвязный")
            print(f"  - УСЛОВИЕ 1: Выполнено (граф сильносвязный)")
            print(f"  - УСЛОВИЕ 2: Выполнено (ограниченность подтверждена)")
            print(f"\nВЫВОД ДЛЯ РАСШИРЕННОЙ СЕТИ: ограниченная и живая -> БЕЗДЕФЕКТНАЯ")

            # Дополнительная проверка: все ли переходы присутствуют в SCC
            all_nodes = set(G_extended.nodes())
            if len(scc_extended[0]) == len(all_nodes):
                print(f"  - ДОПОЛНИТЕЛЬНО: Все состояния в одной SCC")
        else:
            # Проверяем, есть ли доминирующая SCC, содержащая все важные состояния
            all_nodes = set(G_extended.nodes())
            largest_scc = max(scc_extended, key=len)

            print(f"\nАнализ наибольшей SCC:")
            print(f"  - Размер наибольшей SCC: {len(largest_scc)} из {len(all_nodes)} состояний")
            print(f"  - Покрытие: {len(largest_scc)/len(all_nodes)*100:.1f}%")

            # Проверяем, достижима ли наибольшая SCC из всех состояний
            # Для этого строим конденсацию графа
            condensation = nx.condensation(G_extended)

            # Находим компоненты-стоки (терминальные SCC)
            terminal_scc_indices = [node for node in condensation.nodes()
                                   if condensation.out_degree(node) == 0]

            print(f"  - Количество терминальных SCC: {len(terminal_scc_indices)}")

            if len(terminal_scc_indices) == 1:
                print(f"  - УСЛОВИЕ 1: Выполнено (одна терминальная SCC)")
                print(f"  - УСЛОВИЕ 2: Выполнено (ограниченность подтверждена)")
                print(f"\nВЫВОД ДЛЯ РАСШИРЕННОЙ СЕТИ: ограниченная и живая -> БЕЗДЕФЕКТНАЯ")
            else:
                print(f"  - УСЛОВИЕ 1: Не выполнено (несколько терминальных SCC)")
                print(f"  - УСЛОВИЕ 2: Выполнено (ограниченность подтверждена)")
                print(f"\nВЫВОД ДЛЯ РАСШИРЕННОЙ СЕТИ: ограниченная, но не живая")
    else:
        print(f"  - Граф достижимости содержит {len(extended_ts.states)} состояний")
        print(f"  - Возможно, сеть неограниченная или слишком сложная для полного анализа")

        # Пробуем проанализировать первые несколько состояний
        print(f"\nАнализ первых 100 состояний:")
        sample_states = list(extended_ts.states)[:100]

        # Создаем частичный граф для анализа
        G_partial = nx.DiGraph()
        for state in sample_states:
            G_partial.add_node(str(state))

        # Добавляем переходы между этими состояниями
        for transition in extended_ts.transitions:
            if str(transition.from_state) in G_partial and str(transition.to_state) in G_partial:
                G_partial.add_edge(str(transition.from_state), str(transition.to_state))

        # Анализируем частичный граф
        if len(G_partial.nodes()) > 0:
            scc_partial = list(nx.strongly_connected_components(G_partial))
            print(f"  - Количество SCC в выборке: {len(scc_partial)}")

            if len(scc_partial) > 0:
                largest_scc = max(scc_partial, key=len)
                print(f"  - Размер наибольшей SCC в выборке: {len(largest_scc)} состояний")

        print(f"\nВЫВОД ДЛЯ РАСШИРЕННОЙ СЕТИ: требуется дополнительный анализ (возможно неограниченная)")

except Exception as e:
    print(f"Ошибка при построении расширенной сети: {e}")
    import traceback
    traceback.print_exc()
    print("Продолжаем без дополнительного анализа расширенной сети")

## 16. Анализ Directly Follows Graph (DFG)

In [ ]:

from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization

dfg = dfg_discovery.apply(log_for_mining, variant=dfg_discovery.Variants.PERFORMANCE)

print(f"Количество связей между событиями: {len(dfg)}")

gviz_dfg_freq = dfg_visualization.apply(
    dfg,
    log=log_for_mining,
    variant=dfg_visualization.Variants.FREQUENCY,
    parameters={"format": "png"}
)
dfg_visualization.view(gviz_dfg_freq)

gviz_dfg_perf = dfg_visualization.apply(
    dfg,
    log=log_for_mining,
    variant=dfg_visualization.Variants.PERFORMANCE,
    parameters={"format": "png"}
)
dfg_visualization.view(gviz_dfg_perf)

# Сортируем связи по частоте
sorted_dfg = sorted(dfg.items(), key=lambda x: x[1], reverse=True)


print("\n=== Анализ степени связности событий ===")

in_degree = {}
out_degree = {}

for (source, target), count in dfg.items():
    out_degree[source] = out_degree.get(source, 0) + count
    in_degree[target] = in_degree.get(target, 0) + count

print("\nСобытия с наибольшим числом исходящих связей (инициаторы):")
sorted_out = sorted(out_degree.items(), key=lambda x: x[1], reverse=True)
for event, count in sorted_out[:5]:
    print(f"  {event:40}: {count:4} исходящих связей")

print("\nСобытия с наибольшим числом входящих связей (завершители):")
sorted_in = sorted(in_degree.items(), key=lambda x: x[1], reverse=True)
for event, count in sorted_in[:5]:
    print(f"  {event:40}: {count:4} входящих связей")

# Построение DFG как промежуточного шага для Heuristics Miner
# Heuristics Miner использует DFG для вычисления зависимостей
from pm4py.algo.discovery.heuristics.algorithm import HeuristicsNet

heu_net_from_dfg = HeuristicsNet(dfg)

gviz_heu_dfg = hn_visualizer.apply(heu_net_from_dfg)
hn_visualizer.view(gviz_heu_dfg)


# Вычисляем матрицу зависимости для Heuristics Miner
dependency_matrix = heu_net_from_dfg.dependency_matrix

print("Примеры сильных зависимостей между событиями (значение > 0.8):")
strong_dependencies = []

for (source, target), value in dependency_matrix.items():
    if value > 0.8 and source != target:
        strong_dependencies.append(((source, target), value))

strong_dependencies.sort(key=lambda x: x[1], reverse=True)

for i, ((source, target), value) in enumerate(strong_dependencies[:10], 1):
    print(f"{i:2}. {source:30} -> {target:30}: зависимость = {value:.3f}")

# Визуализация DFG с помощью NetworkX для дополнительного анализа
print("\n=== Дополнительный анализ DFG через NetworkX ===")

import networkx as nx

# Создаем граф NetworkX из DFG
G_dfg = nx.DiGraph()

# Добавляем узлы (события)
all_events = set()
for (source, target), count in dfg.items():
    all_events.add(source)
    all_events.add(target)

for event in all_events:
    G_dfg.add_node(event)

# Добавляем ребра с весами (частотами)
for (source, target), count in dfg.items():
    G_dfg.add_edge(source, target, weight=count)

print(f"Граф DFG в NetworkX:")
print(f"  - Узлов (событий): {G_dfg.number_of_nodes()}")
print(f"  - Ребер (последовательностей): {G_dfg.number_of_edges()}")

# Анализ центральности
print("\nАнализ центральности событий в DFG:")

# Степень центральности (degree centrality)
degree_centrality = nx.degree_centrality(G_dfg)
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)

print("События с наибольшей степенью центральности:")
for event, centrality in sorted_degree[:5]:
    print(f"  {event:40}: центральность = {centrality:.3f}")

# Посредническая центральность (betweenness centrality)
betweenness_centrality = nx.betweenness_centrality(G_dfg)
sorted_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)

print("\nСобытия с наибольшей посреднической центральностью:")
for event, centrality in sorted_betweenness[:5]:
    print(f"  {event:40}: центральность = {centrality:.3f}")